# King County advanced EDA

This notebook follows some advanced exploratory data analaysis ideas outlined in Michael P. Notters ["Advanced exploratory data analysis (EDA)"](https://miykael.github.io/blog/2022/advanced_eda/) and is used as an practice approach to improve my EDA.

It uses some tools to get a quick handle on tabular data, but is in no way a complete approach.

In this notebook we first look int the structure of our data, followed by the quality and content.

1. **Structure**: general exploration of shape, data types, etc.
2. **Quality**: informaton about missing data, duplicate entries, outliers (unwanted entries)
3. **Content**: more in-depth analysis to understand our feature values and how they relate to each other


Beforehand however we need to load the data. For this purpose we are using pandas.

In [113]:
import pandas as pd

# load the data into a pandas data frame using the pd.read_csv method
df = pd.read_csv('data/King_County_House_prices_dataset.csv')

### 1. Structure Analysis

As mentioned, before we are looking into the quality and Content of our data, we start out with a very basic analysis of the structure.
The first step is to find out how many observations and features we have.

In [114]:
# display size of the dataframe
df.shape

(21597, 21)

With this we know that our dataset contains roughl 21k observations and 21 column features and we shortly have a look at the column names to get a first glance at the features.

In [115]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

Next, we want to understand with how many different data types we are dealing.

In [116]:
# display how often the different data types are represented within our dataset
df.dtypes.value_counts()

int64      11
float64     8
object      2
dtype: int64

#### 1.1 Structure of non-numerical features

We are dealing with numerical data (int and float) as well as with 2 non-numerical features. We start by looking into the 2 non-numerical columns in our dataset.

In [117]:
# display the first 5 entries of our non-numerical features
df.select_dtypes(exclude='number').head()

,date,sqft_basement
0,10/13/2014,0.0
1,12/9/2014,400.0
2,2/25/2015,0.0
3,12/9/2014,910.0
4,2/18/2015,0.0


Even though sqft_basement is a numerical feature it was stored as a non-numerical one. Quite similar with date, which is ideally stored as a dateformat for further analysis.
Thus, in the next step we look more in detail into this two features and try to convert them to their ideal data type. The direct convertion using .astype is not working as currently there are some values existing, which cannot be converted automatically.

Thhus, in order to check if the sqft_basement contains only numerical data and can be converted, we are extracting the individual values of the sqft_basement series and parsing it into a list.

In [118]:
# create a lit of unique values available in the sqft_basement series
value_list = set(df['sqft_basement'].values)

# check for values not containing numbers and the decimal seperator
for element in value_list:
    if re.search('[^0123456789.]', element):
        print(element)

?


Some observations contain a question mark instead of a number. As we cannot convert the ? into a float or int, we investigate first how many observations hold this value. Based on this, we can either drop all impacted observations or impede the data with a different value.

In [119]:
len(df[df['sqft_basement'] == '?'])

454

With 454 entries the amount is to high as to drop the observations. Accordingly, we impede the missing data. We could either choose the median or mean to fill in. However, in this case, we treat the missing value as not having a basement size and fill it with 0 instead.

In [120]:
# replace the question mark with 0.0
df['sqft_basement'] = df['sqft_basement'].replace('?', 0.0)

# double check the amount of observations containing a question mark
len(df[df['sqft_basement'] == '?'])

0

From looking at the column names earlier on, we saw that we have multiple columns describing the sqft size of property/ground. Thus, we shortly check if we are dealing with int or float values for the other types and try to convert the sqft_basement in accordance.

In [121]:
# check the data type of the sqft features
df[['sqft_living', 'sqft_lot', 'sqft_above']].dtypes

sqft_living    int64
sqft_lot       int64
sqft_above     int64
dtype: object

In [122]:
# check the first entries of the sqft_basement again
df['sqft_basement'].head()

0      0.0
1    400.0
2      0.0
3    910.0
4      0.0
Name: sqft_basement, dtype: object

As we can see we are dealing with float like values which are currently cast as a string. These values cannot automatically cast into int values due due the decimal values.


In [127]:
# convert the object type values into float and afterwards to int
df['sqft_basement'] = df['sqft_basement'].astype('float').astype('int')
df['sqft_basement'].dtypes

dtype('int64')